### 事前準備（ライブラリのインポートなど）

In [21]:
# 標準ライブラリ
import re
from tqdm import tqdm
import pandas as pd
# import matplotlib.pyplot as plt
import numpy as np

# 自然言語系のライブラリ
import sudachipy
from gensim.summarization.bm25 import BM25

import warnings
warnings.simplefilter('ignore')


### 形態素分類

In [22]:
# ストップワード（処理対象外の単語）の読み込み
!wget -nv -N -P "../data/" "http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt"

with open("../data/Japanese.txt","r") as f:
    stop_words = f.read().split("\n")

In [23]:
#Neologdによるトーカナイザー(リストで返す関数・名詞のみ)
def sudachi_tokenizer(text: str) -> list:

    # 正規表現による文章の修正
    replaced_text = text.lower() # 全て小文字へ変換
    replaced_text = re.sub(r'[【】]', '', replaced_text) # 【】の除去
    replaced_text = re.sub(r'[（）()]', '', replaced_text) # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', '', replaced_text) # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text) # メンションの除去
    replaced_text = re.sub(r'\d+\.*\d*', '', replaced_text) #数字を0にする
    replaced_text = re.sub(r'[*、％]', '', replaced_text) # 記号の除去

    # sudachi tokenize
    tokenizer_obj = sudachipy.Dictionary().create()
    mode = sudachipy.Tokenizer.SplitMode.C

    parsed_lines = tokenizer_obj.tokenize(replaced_text, mode)

    # 名詞かつ固有名詞ではないものに絞り込み
    token_list = [t.surface() for t in parsed_lines if t.part_of_speech()[0] == "名詞" and t.part_of_speech()[1] != "固有名詞"]

    # stop wordsの除去
    token_list = [t for t in token_list if t  not in stop_words]

    # ひらがなのみの単語を除く
    kana_re = re.compile("^[ぁ-ゖ]+$")
    token_list = [t for t in token_list if not kana_re.match(t)]

    return token_list

In [28]:
class best_match:
    def __init__(self):
        self.tokenizer = sudachi_tokenizer

    #前処理
    def pre_process(self, docs):
        self.docs = docs
        corpus = [self.wakachi(doc) for doc in self.docs]
        self.bm25_ = BM25(corpus)

    #クエリとの順位付け
    def ranking(self, query):
        wakachi_query = self.wakachi(query)
        self.scores = self.bm25_.get_scores(wakachi_query)

    #分かち書き
    def wakachi(self, doc):
        return list(self.tokenizer(doc))

    #上位n件を抽出
    def select_docs(self, num):
        docs_dict = dict(zip(self.scores, self.docs))
        docs_dict = dict(sorted(docs_dict.items(), reverse = True))
        print("\n・検索結果")
        i = 0
        for key, value in docs_dict.items():
            print(round(key, 3), value)
            i += 1
            if i == num: break

if __name__ == "__main__":
    query = "インドカレー屋で提供されているラッシーは、とても美味しい。"
    docs = ["カレーよりもシチューが好きだ。",
            "ガンジス川を見るためにインドに来た。",
            "カレーが好きだ。中でも、インドカレーが一番好きだ。",
            "自宅で作ったラッシーも美味しい。",
            "欧風カレーとインドカレーは全くの別物だが、どちらも美味しい。",
            "インドカレーが好きだ。"]
    while True:
        try:
            num = int(input("検索数を自然数で入力してください:"))
            if num <= 0:
                print("0より大きな数字を入力してください。")
            elif num < len(docs):
                break
            else:
                print("文書数より多い数字が入力されています。")
        except Exception:
            print("数字以外のテキストが入力されています。")

    print("クエリ:", query)
    inst_BM = best_match()
    inst_BM.pre_process(docs)
    inst_BM.ranking(query)
    inst_BM.select_docs(num)


クエリ: ガンジス川は汚い

・検索結果
1.633 ガンジス川を見るためにインドに来た。
0.0 インドカレーが好きだ。
